# BUILDING A TABLE OF FIELD GALS WITH CASSATA AND/OR ZURICH HALF-LIGHT RADIUS COUNTERPART AND ADDING A NEW COLUMN WHICH IS EITHER ZURICH RADIUS OR BIAS CORRECTED CASSATA RADIUS

## Getting data

In [2]:
pathdata = "outputs/"
data     = ["matching_fieldGals_with_Cassata_and_Zurich_acc_1_arcsec.vot", 
            "matching_fieldGals_with_Cassata_acc_1_arcsec.vot"]

#Checking that file format is correct
for name in data:
    voTag = is_VOtable(pathdata+name)
    if voTag:        
        fullFileName = pathdata + name
        #Retrieving the data
        table = parse(fullFileName)
        full  = table.get_first_table()
        
        print("Size of", name, "is", full.array.shape[0], "\n")
    else:
        exit("Exiting")

The file outputs/matching_fieldGals_with_Cassata_and_Zurich_acc_1_arcsec.vot is a VOtable, right ? True


Size of matching_fieldGals_with_Cassata_and_Zurich_acc_1_arcsec.vot is 186 

The file outputs/matching_fieldGals_with_Cassata_acc_1_arcsec.vot is a VOtable, right ? True


Size of matching_fieldGals_with_Cassata_acc_1_arcsec.vot is 298 



In [3]:
matchWithCassata       = parse(pathdata+data[1]).get_first_table().array
matchWithZurichCassata = parse(pathdata+data[0]).get_first_table().array

#Checking that the matching procedure did not duplicate galaxies
checkDupplicates([matchWithZurichCassata, matchWithCassata], 
                 names=["matching_fieldGals_with_Cassata_and_Zurich_acc_1_arcsec.vot", 
                        "matching_fieldGals_with_Cassata_acc_1_arcsec.vot"])

All the galaxies are only listed once in the catalog matching_fieldGals_with_Cassata_and_Zurich_acc_1_arcsec.vot
All the galaxies are only listed once in the catalog matching_fieldGals_with_Cassata_acc_1_arcsec.vot


In [4]:
#Converting to an astropy table for simplicity
tableMatchBoth = Table(matchWithZurichCassata)        
tableMatchCass = Table(matchWithCassata)

#Checking that the maximum angular separation is less than 1 arcsec in the matching tables
printSimpleStat([tableMatchBoth['Separation_ZURICH'], 
                 tableMatchCass['Separation_CASSATA']])

#Remove galaxies which have wrong ID_Laigle_16 values (9999)
tableMatchBoth = tableMatchBoth[tableMatchBoth['ID_Laigle_16'] != 9999]
tableMatchBoth = tableMatchBoth[tableMatchBoth['CONFID'] > 1]
tableMatchCass = tableMatchCass[tableMatchCass['ID_Laigle_16'] != 9999]
tableMatchCass = tableMatchCass[tableMatchCass['CONFID'] > 1]

findWhereIsValue([tableMatchCass['ID_Laigle_16'], tableMatchBoth['ID_Laigle_16']], 9999)
findWhereIsValue([tableMatchCass['CONFID'], tableMatchBoth['CONFID']], 1)

#Remove the galaxy counted twice in the arrays
uniqueArr([tableMatchBoth, tableMatchCass], [tableMatchBoth['ID_Laigle_16'], tableMatchCass['ID_Laigle_16']])

print("\nNumber of galaxies in matching catalog with Cassata:", np.shape(tableMatchCass)[0], 
      "\nNumber of galaxies in matching catalog with Cassata & Zurich:", np.shape(tableMatchBoth)[0])

Stat for catalog number 0 :
Maximum separation is 0.8778970454248154.
Mean separation is 0.10371229092642334.
Median separation is 0.06769009751877036.
1st quantile is 0.042612085489821414.
3rd quantile is 0.12115790194733166.
Stat for catalog number 1 :
Maximum separation is 0.8768483517534824.
Mean separation is 0.10767987793148323.
Median separation is 0.0709917623259244.
1st quantile is 0.0429616639175216.
3rd quantile is 0.1264333202418157.


/home/wilfried/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:688: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


No value 9999 found within array number 0
No value 9999 found within array number 1
No value 1 found within array number 0
No value 1 found within array number 1

Number of galaxies in matching catalog with Cassata: 261 
Number of galaxies in matching catalog with Cassata & Zurich: 176


In [5]:
#Get a subtable with galaxies having Cassata radius only
mask            = np.in1d(tableMatchCass['ID_Laigle_16'], tableMatchBoth['ID_Laigle_16'], 
                        assume_unique=True, invert=True)
tableCassOnly   = tableMatchCass[mask]

#Keep list of fields which appear in Cassata+Zurich matching catalog but not in Cassata matching catalog
m               = np.in1d(tableMatchBoth.dtype.names, tableCassOnly.dtype.names, assume_unique=True, invert=True)
listFieldsToAdd = np.asarray(tableMatchBoth.dtype.names)[m]

#Adding Zurich columns (default value=-999) to the Table with galaxies in Cassata catalog only
sz = np.shape(tableCassOnly)[0]
for field in listFieldsToAdd:
    tableCassOnly[field] = np.array([-9999]*sz, dtype=tableMatchBoth.dtype[field])

#Correcting biais in Cassata radius
offset         = -3.495e-1
A              = 1.95e-3
R_half_CASSATA = tableCassOnly['R_HALF_CASSATA']/(1+offset+A*tableCassOnly['R_HALF_CASSATA'])

#Creating new column in both tables
newField                 = 'Corrected_radius'
tableMatchBoth[newField] = tableMatchBoth['R50_ZURICH']
tableCassOnly[newField]  = R_half_CASSATA

## Concatenating tables

In [6]:
newTable = vstack([tableMatchBoth, tableCassOnly], join_type='exact')

## Writing the full table in a new file

In [7]:
path       = 'outputs/'
outputFile = 'matching_fieldGals_Cassata_and_Zurich_corrected_radius_v2.vot'
# write_array_to_vot(newTable, path+outputFile, isTable=True)